# TauREx-CuPy

[TauREx-CuPy](https://pypi.org/project/taurex-cupy/) is a plugin for TauREx. That allows for drop-in replacement classes that give us CUDA-accelerated radiative transfer!!

Lets setup:

In [1]:
!pip install taurex corner

!mkdir xsec/

#### Exomol H2O ####
! wget https://exomol.com/db/H2O/1H2-16O/POKAZATEL/1H2-16O__POKAZATEL__R15000_0.3-50mu.xsec.TauREx.h5 -P ./xsec/
#### Exomol CH4 ####
! wget https://exomol.com/db/CH4/12C-1H4/MM/12C-1H4__MM.R15000_0.3-50mu.xsec.TauREx.h5 -P ./xsec
#### Exomol NH3 ####
! wget https://exomol.com/db/NH3/14N-1H3/CoYuTe/14N-1H3__CoYuTe.R15000_0.3-50mu.xsec.TauREx.h5 -P ./xsec/
#### Exomol CO2 ####
! wget https://exomol.com/db/CO2/12C-16O2/UCL-4000/12C-16O2__UCL-4000.R15000_0.3-50mu.xsec.TauREx.h5 -P ./xsec/

!mkdir cia

!wget https://hitran.org/data/CIA/H2-H2_2011.cia -P ./cia
!wget https://hitran.org/data/CIA/H2-He_2011.cia -P ./cia

mkdir: cannot create directory ‘xsec/’: File exists
--2025-02-12 22:08:16--  https://exomol.com/db/H2O/1H2-16O/POKAZATEL/1H2-16O__POKAZATEL__R15000_0.3-50mu.xsec.TauREx.h5
Resolving exomol.com (exomol.com)... 128.40.3.60
Connecting to exomol.com (exomol.com)|128.40.3.60|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 365310072 (348M)
Saving to: ‘./xsec/1H2-16O__POKAZATEL__R15000_0.3-50mu.xsec.TauREx.h5.1’

1H2-16O__POKAZATEL_ 100%[===================>] 348.39M  18.6MB/s    in 20s     

2025-02-12 22:08:36 (17.7 MB/s) - ‘./xsec/1H2-16O__POKAZATEL__R15000_0.3-50mu.xsec.TauREx.h5.1’ saved [365310072/365310072]

--2025-02-12 22:08:37--  https://exomol.com/db/CH4/12C-1H4/MM/12C-1H4__MM.R15000_0.3-50mu.xsec.TauREx.h5
Resolving exomol.com (exomol.com)... 128.40.3.60
Connecting to exomol.com (exomol.com)|128.40.3.60|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 365310264 (348M)
Saving to: ‘./xsec/12C-1H4__MM.R15000_0.3-50mu.xsec.TauREx.h5.1

Grab some data:

In [2]:
!wget https://raw.githubusercontent.com/ucl-exoplanets/taurex3/refs/heads/main/examples/parfiles/quickstart.dat

--2025-02-12 22:09:48--  https://raw.githubusercontent.com/ucl-exoplanets/taurex3/refs/heads/main/examples/parfiles/quickstart.dat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10752 (10K) [text/plain]
Saving to: ‘quickstart.dat.1’

quickstart.dat.1    100%[===================>]  10.50K  --.-KB/s    in 0s      

2025-02-12 22:09:48 (103 MB/s) - ‘quickstart.dat.1’ saved [10752/10752]



Setup our caches:

In [3]:
from taurex.cache import OpacityCache, CIACache

OpacityCache().set_opacity_path('./xsec/')
CIACache().set_cia_path('./cia/')

Now lets install the plugin!

In [4]:
!pip install taurex-cupy --no-deps

# Building the CUDA model

Lets take the standard model from before:

In [5]:
from taurex.temperature import Isothermal
from taurex.planet import Planet
from taurex.stellar import BlackbodyStar
from taurex.chemistry import TaurexChemistry, ConstantGas
from taurex.model import TransmissionModel
from taurex.contributions import AbsorptionContribution, RayleighContribution, CIAContribution

isothermal = Isothermal(T=1500.0)

chemistry = TaurexChemistry(fill_gases=['H2','He'],ratio=0.172)

chemistry.addGas(ConstantGas('H2O',mix_ratio=1.2e-4))
chemistry.addGas(ConstantGas('CH4',mix_ratio=1.2e-5))
chemistry.addGas(ConstantGas('NH3',mix_ratio=1.2e-6))
chemistry.addGas(ConstantGas('CO2',mix_ratio=1.2e-6))

tm_cpu = TransmissionModel(planet=Planet(planet_radius=1.0,planet_mass=1.0),
                       temperature_profile=isothermal,
                       chemistry=chemistry,
                       star=BlackbodyStar(temperature=5700.0,radius=1.0),
                       atm_min_pressure=1e-0,
                       atm_max_pressure=1e6,
                       nlayers=30)
tm_cpu.add_contribution(AbsorptionContribution())
tm_cpu.add_contribution(CIAContribution(cia_pairs=['H2-H2','H2-He']))
tm_cpu.add_contribution(RayleighContribution())
tm_cpu.build()

The CUDA version replaces the ``Model`` and ``Contributions`` with ``Cuda`` varients like so:

In [6]:
from taurex_cupy import TransmissionCudaModel
from taurex_cupy import AbsorptionCuda, CIACuda, RayleighCuda

tm_gpu = TransmissionCudaModel(planet=Planet(planet_radius=1.0,planet_mass=1.0),
                       temperature_profile=isothermal,
                       chemistry=chemistry,
                       star=BlackbodyStar(temperature=5700.0,radius=1.0),
                       atm_min_pressure=1e-0,
                       atm_max_pressure=1e6,
                       nlayers=30)
tm_gpu.add_contribution(AbsorptionCuda())
tm_gpu.add_contribution(CIACuda(cia_pairs=['H2-H2','H2-He']))
tm_gpu.add_contribution(RayleighCuda())
tm_gpu.build()

Lets assess the performance:

In [7]:
# Warmup (load files etc.)
tm_cpu.model()
tm_gpu.model()

%timeit tm_cpu.model()
%timeit tm_gpu.model()

482 ms ± 9.68 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
32.6 ms ± 3.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Lets try setting up a retrieval as before:

In [8]:
from taurex.spectrum import ObservedSpectrum
from taurex.optimizer import NestleOptimizer

obs = ObservedSpectrum("./quickstart.dat")

opt = NestleOptimizer(observed=obs, model=tm_gpu, num_live_points=100)
opt.set_model(tm_gpu)

In [9]:
from taurex.core.priors import Uniform, LogUniform

opt.enable_fit("T")
opt.set_prior("T", Uniform(bounds=(1000.0, 2000.0)))

opt.enable_fit("planet_radius")
opt.set_prior("planet_radius", Uniform(bounds=(0.5,2.0)))

opt.enable_fit("H2O")
opt.set_prior("H2O", LogUniform(bounds=(-12,-3)))

opt.enable_fit("CH4")
opt.set_prior("CH4", LogUniform(bounds=(-12,-3)))

Now lets fit:

In [ ]:
solution = opt.fit()

it=  1401 logz=1695.662728

Lets loop and plot each solution

In [ ]:
import matplotlib.pyplot as plt
obin = obs.create_binner()

for sol,optimized_map,optimized_value,values in opt.get_solution():
    opt.update_model(optimized_map)
    plt.figure()
    plt.errorbar(obs.wavelengthGrid,obs.spectrum,obs.errorBar,label='Obs')
    plt.plot(obs.wavelengthGrid,obin.bin_model(tm_gpu.model(obs.wavenumberGrid))[1],label='TM')
    plt.legend()
    plt.show()

Lets plot the corners